# Arthur LangChain integration

[Arthur](https://www.arthur.ai/) is a model monitoring and observability platform.

This notebook shows how to register LLMs (chat and non-chat) as models with the Arthur platform. Then we show how to set up langchain LLMs with an Arthur callback that will automatically log model inferences to Arthur.

For more information about how to use the Arthur SDK, visit our [docs](http://docs.arthur.ai), in particular our [model onboarding guide](https://docs.arthur.ai/user-guide/walkthroughs/model-onboarding/index.html)

In [21]:
from langchain.callbacks import ArthurCallbackHandler
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI, ChatAnthropic
from langchain.schema import HumanMessage
from langchain.llms import OpenAI, Cohere, HuggingFacePipeline

In [3]:
from arthurai import ArthurAI
from arthurai.common.constants import InputType, OutputType, Stage, ValueType
from arthurai.core.attributes import ArthurAttribute, AttributeCategory

# ArthurModel for chatbot with only input text and output text attributes

Connect to Arthur client

In [4]:
arthur_url = "https://app.arthur.ai"
arthur_login = "your-username-here"
arthur = ArthurAI(url=arthur_url, login=arthur_login)

Before you can register model inferences to Arthur, you must have a registered model with an ID in the Arthur platform. We will provide this ID to the ArthurCallbackHandler.

You can register a model with Arthur here in the notebook using this `register_chat_llm()` function. This function returns the ID of the model saved to the platform. To use the function, uncomment `arthur_model_chatbot_id = register_chat_llm()` in the cell below.

In [5]:
def register_chat_llm():

    arthur_model = arthur.model(
        display_name="LangChainChat",
        input_type=InputType.NLP,
        output_type=OutputType.TokenSequence
    )

    arthur_model._add_attribute_to_model(ArthurAttribute(
        name="my_input_text",
        stage=Stage.ModelPipelineInput,
        value_type=ValueType.Unstructured_Text,
        categorical=True,
        is_unique=True
    ))
    arthur_model._add_attribute_to_model(ArthurAttribute(
        name="my_output_text",
        stage=Stage.PredictedValue,
        value_type=ValueType.Unstructured_Text,
        categorical=True,
        is_unique=False,
    ))
    
    return arthur_model.save()
# arthur_model_chatbot_id = register_chat_llm()

Alternatively, you can set the `arthur_model_chatbot_id` variable to be the ID of your model on your [model dashboard](https://app.arthur.ai/)

In [1]:
arthur_model_chatbot_id = "your-model-id-here"

This function creates a Langchain chat LLM with the ArthurCallbackHandler to log inferences to Arthur. We provide our `arthur_model_chatbot_id`, as well as the Arthur url and login we are using.

In [7]:
def make_langchain_chat_llm(chat_model=ChatOpenAI):
    if chat_model not in [ChatOpenAI, ChatAnthropic]:
        raise ValueError("For this notebook, use one of the chat models imported from langchain.chat_models")
    return chat_model(
        streaming=True, 
        temperature=0.1,
        callbacks=[
            StreamingStdOutCallbackHandler(), 
            ArthurCallbackHandler.from_credentials(arthur_model_chatbot_id, arthur_url=arthur_url, arthur_login=arthur_login)
        ])


In [8]:
chat_llm = make_langchain_chat_llm()

Run the chatbot (it will save the chat history in the `history` list so that the conversation can reference earlier messages)

Type `q` to quit

In [9]:
def run_langchain_chat_llm(llm):
    history = []
    while True:
        user_input = input("\n>>> input >>>\n>>>: ")
        if user_input == 'q': break
        history.append(HumanMessage(content=user_input))
        history.append(llm(history))

In [10]:
run_langchain_chat_llm(chat_llm)

# ArthurModel with input text, output text, token likelihoods, finish reason, and amount of token usage attributes

This function registers an LLM with additional metadata attributes to log to Arthur with each inference

As above, you can register your callback handler for an LLM using this function here in the notebook or by pasting the ID of an already-registered model from your [model dashboard](https://app.arthur.ai/)

In [11]:
def register_llm():

    arthur_model = arthur.model(
        display_name="LangChainLLM",
        input_type=InputType.NLP,
        output_type=OutputType.TokenSequence
    )
    arthur_model._add_attribute_to_model(ArthurAttribute(
        name="my_input_text",
        stage=Stage.ModelPipelineInput,
        value_type=ValueType.Unstructured_Text,
        categorical=True,
        is_unique=True
    ))
    arthur_model._add_attribute_to_model(ArthurAttribute(
        name="my_output_text",
        stage=Stage.PredictedValue,
        value_type=ValueType.Unstructured_Text,
        categorical=True,
        is_unique=False,
        token_attribute_link="my_output_likelihoods"
    ))
    arthur_model._add_attribute_to_model(ArthurAttribute(
        name="my_output_likelihoods",
        stage=Stage.PredictedValue,
        value_type=ValueType.TokenLikelihoods,
        token_attribute_link="my_output_text"
    ))
    arthur_model._add_attribute_to_model(ArthurAttribute(
        name="finish_reason",
        stage=Stage.NonInputData,
        value_type=ValueType.String,
        categorical=True,
        categories=[
            AttributeCategory(value='stop'),
            AttributeCategory(value='length'),
            AttributeCategory(value='content_filter'),
            AttributeCategory(value='null')
        ]
    ))
    arthur_model._add_attribute_to_model(ArthurAttribute(
        name="prompt_tokens",
        stage=Stage.NonInputData,
        value_type=ValueType.Integer
    ))
    arthur_model._add_attribute_to_model(ArthurAttribute(
        name="completion_tokens",
        stage=Stage.NonInputData,
        value_type=ValueType.Integer
    ))
    arthur_model._add_attribute_to_model(ArthurAttribute(
        name="duration",
        stage=Stage.NonInputData,
        value_type=ValueType.Float
    ))
    
    return arthur_model.save()

In [12]:
arthur_model_llm_id = "your-model-id-here"

These functions create Langchain LLMs with the ArthurCallbackHandler to log inferences to Arthur.

There are small differences in the underlying Langchain integrations with these libraries and the available metadata for model inputs & outputs

In [23]:
def make_langchain_openai_llm():
    return OpenAI(
        temperature=0.1,
        model_kwargs = {'logprobs': 3},
        callbacks=[
            ArthurCallbackHandler.from_credentials(arthur_model_llm_id, arthur_url=arthur_url, arthur_login=arthur_login)
        ])

def make_langchain_cohere_llm():
    return Cohere(
        temperature=0.1,
        callbacks=[
            ArthurCallbackHandler.from_credentials(arthur_model_chatbot_id, arthur_url=arthur_url, arthur_login=arthur_login)
        ])

def make_langchain_huggingface_llm():
    llm = HuggingFacePipeline.from_model_id(
        model_id="bert-base-uncased", 
        task="text-generation", 
        model_kwargs={"temperature":2.5, "max_length":64})
    llm.callbacks = [
        ArthurCallbackHandler.from_credentials(arthur_model_chatbot_id, arthur_url=arthur_url, arthur_login=arthur_login)
    ]
    return llm

In [24]:
openai_llm = make_langchain_openai_llm()

In [25]:
cohere_llm = make_langchain_cohere_llm()

In [2]:
huggingface_llm = make_langchain_huggingface_llm()

Run the LLM (each completion is independent, no chat history is saved as we were doing above with the chat llms)

Type `q` to quit

In [17]:
def run_langchain_llm(llm):
    while True:
        print("Type your text for completion:\n")
        user_input = input("\n>>> input >>>\n>>>: ")
        if user_input == 'q': break
        print(llm(user_input), "\n================\n")

In [18]:
run_langchain_llm(openai_llm)

In [19]:
run_langchain_llm(cohere_llm)

In [1]:
run_langchain_llm(huggingface_llm)